# Particles in a dual tree

A very simple demonstration how to use the pidt (particle in dual tree) scheme
within Peano 4.

Create a project and configure it to end up in a subnamespace (and thus subdirectory). 

In [1]:
import os

import peano4
import peano4.dastgen2
import peano4.toolbox
import peano4.toolbox.particles
import dastgen2


DaStGen 2 (C) www.peano-framework.org
Peano 4 (C) www.peano-framework.org


In [2]:
project = peano4.Project( ["examples", "particles"], "particles", "." )

## Prepare code

Peano's API does not know which settings to use on the present system. To make it copy/clone the settings identified by ./configure, we ask it to parse the generated configuration scripts. 


In [3]:
project.output.makefile.parse_configure_script_outcome( "../.." )

parse configure outcome ../../src/Makefile to extract compile settings
used C++ compiler is mpiicpc
used Fortran compiler is ifort


## Model the particle

We create the particle through the toolbox. A particle by definition has a position. So the only thing we have to add is a velocity. We use Peano's double array to have full support of Peano's vector classes, and instead of saying that this array had two or three entries, we parameterise it through the symbol Dimensions.

In [4]:
particle  = peano4.toolbox.particles.Particle( "Particle" )
particle.data.add_attribute( peano4.dastgen2.Peano4DoubleArray("v","Dimensions") )
particles = peano4.toolbox.particles.ParticleSet( particle )

This particle has to be added to the project. This way the project knows that we are working with this guy. We associate the particle with the vertices, as we work with the pidt (particle in dual tree) algorithm. 

In [5]:
project.datamodel.add_global_object(particle)
project.datamodel.add_vertex(particles)

## Model the algorithmic steps we rely on

When we model the algorithmic steps we rely on, we use a number of pre-defined action sets from the particle toolbox. First and foremost, we use an analysed tree grammar, i.e. some marking mechanism for the tree nodes. This helps us to guide the AMR - which is the second ingredient that we use. Finally we use the plotting routines for particles from the toolbox.

In [6]:
particle_tree_analysis = peano4.toolbox.particles.ParticleTreeAnalysis(particles)
project.datamodel.add_cell(particle_tree_analysis.cell_marker)   # read docu of ParticleTreeAnalysis

### Grid/experiment setup

The first thing we will do is to create a set of particles and add them to the grid.

In [7]:
create_grid = peano4.solversteps.Step( "CreateGrid" )
create_grid.use_vertex(particles)
create_grid.use_cell(particle_tree_analysis.cell_marker)
create_grid.add_action_set( peano4.toolbox.particles.ParticleDensityAMR(particle_tree_analysis) )
create_grid.add_action_set( particle_tree_analysis )
project.solversteps.add_step(create_grid)

## Move particles around

We'll just give each particle a random position and velocity and move them around following this velocity. That is, we don't make the particles interact with each other et al.

In [8]:
move_particles = peano4.solversteps.Step( "MoveParticles" )
move_particles.use_vertex(particles)
move_particles.use_cell(particle_tree_analysis.cell_marker)
move_particles.add_action_set( peano4.toolbox.particles.ParticleDensityAMR(particle_tree_analysis) )
move_particles.add_action_set( particle_tree_analysis )
project.solversteps.add_step(move_particles)

### Plots setup

I will have to plot the outcome (or any snapshot). With particles, each of my dumps produces two types of files: particle data and mesh data. For the mesh, I rely on the toolbox's grid plotting. The plot is the step where we don't need the AMR criterion - the particles don't move here after all. We however run the tree analysis, so it is up-to-date for the next step.

In [9]:
print_solution = peano4.solversteps.Step( "Plot" )
print_solution.use_vertex(particles)
print_solution.use_cell(particle_tree_analysis.cell_marker)
print_solution.remove_all_actions()
print_solution.add_action_set( particle_tree_analysis )
print_solution.add_action_set( peano4.toolbox.PlotGridInPeanoBlockFormat( "grid", None ) )
print_solution.add_action_set( peano4.toolbox.particles.PlotParticlesInVTKFormat( "particles", particles ) )
project.solversteps.add_step(print_solution)


## Generate the actual C++ code

Standard triad of operations. You can skip the first two steps if you want as the script then will automatically invoke the previous steps. The other way round, it is always admissible to only generate stuff, e.g., but to build and run the project through a command line.

In [10]:
project.generate()
project.build()

generate all code ...
user has to modify class CreateGrid in actions directory manually 
user has to modify class MoveParticles in actions directory manually 
generated particles-main.cpp
write ./Makefile
write ./vertexdata/ParticleSet.h (generated from template /home/tobias/git/Peano/python/peano4/toolbox/particles/ParticleSet.template.h)
write ./vertexdata/ParticleSet.cpp (generated from template /home/tobias/git/Peano/python/peano4/toolbox/particles/ParticleSet.template.cpp)
write ./observers/DataRepository.h
write ./observers/DataRepository.cpp
write ././observers/CreateGrid2Dummy1.h
write ././observers/CreateGrid2Dummy1.cpp
write ././observers/CreateGrid2peano4_toolbox_particles_ParticleTreeAnalysis2.h
write ././observers/CreateGrid2peano4_toolbox_particles_ParticleTreeAnalysis2.cpp
write ././observers/CreateGrid.h
write ././observers/CreateGrid.cpp
write ././observers/MoveParticles2Dummy1.h
write ././observers/MoveParticles2Dummy1.cpp
write ././observers/MoveParticles2peano4_tool

## Implement the actual code

Don't forget to insert

<pre>
#include "../globaldata/Particle.h"
</pre>

## Run code

In [11]:
#success = project.run( args=[], prefix=["mpirun", "-n", "1"] )